Importing primary dependencies for data fetch and cleanup

In [ ]:
import pandas as pd
import numpy as np
import re

from pandas import read_csv

Various constants and data fetch

In [ ]:
max_features=10240
max_len=2048
test_prop=0.3
epoch_count=100
batch_size=512

In [ ]:
fake=read_csv('https://raw.githubusercontent.com/MobiusYeeitus/FakeNewsDetection/master/datasets/set1/fake.csv')
real=read_csv('https://raw.githubusercontent.com/MobiusYeeitus/FakeNewsDetection/master/datasets/set1/true.csv')
data2=read_csv('https://raw.githubusercontent.com/MobiusYeeitus/FakeNewsDetection/master/datasets/cleaned/ds2.csv')

In [ ]:
fake.sample(4)

,title,text,subject,date
8620,Bundy Terrorists Disturb Native American Arti...,Ammon Bundy and his band of gun-toting domesti...,News,"January 21, 2016"
19859,IRANIAN SCIENTIST HANGED For “Revealing [Nucle...,Reckless Hillary couldn t have cared less abou...,left-news,"Oct 6, 2016"
11721,HUNDREDS RALLY After Atheist Group Forces Mayo...,The group called Freedom from Religion has bee...,politics,"Feb 6, 2017"
5871,Trump Accuses American Soldiers Of Stealing M...,From the very outset of Trump s improbable pol...,News,"June 14, 2016"


In [ ]:
real.sample(4)

,title,text,subject,date
18248,Late UK PM Heath had questions to answer over ...,"SWINDON, England (Reuters) - Former British pr...",worldnews,"October 5, 2017"
19916,"Turkey, Iran and Russia to deploy observers ar...","ANKARA/ASTANA (Reuters) - Russia, Iran and Tur...",worldnews,"September 15, 2017"
7226,Congressional panel calls for U.S. women's his...,WASHINGTON (Reuters) - A congressionally-appoi...,politicsNews,"November 16, 2016"
16481,Thailand kicks off sumptuous funeral of King B...,BANGKOK (Reuters) - Thailand on Wednesday mark...,worldnews,"October 25, 2017"


Compressed every cleanup job into the *clean_csv* function

In [ ]:
def clean_string(string): #function to clean a string | to be used in clean_csv
    string=string.lower() #convert string to lowercase
    string=re.sub('\n|\r|\t','',string) #cleaning of whitespaces
    string=re.sub(r'[^\w\s]+','',string) #cleaning of symbols
    return string
def clean_csv(df,identity): #cleaning the CSV file
    print('Cleaning CSV.')
    df=df.dropna() #drop all tuples with empty attributes
    length=[]
    [length.append(len(str(text))) for text in df["text"]] #calculate length of an article
    df['length'] = length #saves the length of the artivle in a new column
    df = df.drop(df['text'][df['length'] < 120].index, axis = 0) #drop all tuples with less than 120 letters
    df['text']=df['title']+" "+df['text'] #append text to title
    vfunc=np.vectorize(clean_string) #vectorize th clean_string function for use in pandas df
    df['text']=vfunc(df['text']) #apply the vectorized function to further clean the data
    df['identity']=identity #add the identity (T/F) or (0/1) in a new attribute to the data
    df.reset_index() #reset all indexes of the dataset
    print('Cleaning completed.')
    return df

In [ ]:
fake1=fake.iloc[:int(len(fake)/2),:]
fake2=fake.iloc[int(len(fake)/2):,:]

In [ ]:
fake1=clean_csv(fake1,0)

Cleaning CSV.
Cleaning completed.


In [ ]:
fake2=clean_csv(fake2,0)

Cleaning CSV.
Cleaning completed.


In [ ]:
fake=pd.concat([fake1,fake2])

In [ ]:
fake.sample(4)

,title,text,subject,date,length,identity
6693,Watch Chris Christie Awkwardly Dance At Sprin...,watch chris christie awkwardly dance at sprin...,News,"April 26, 2016",2498,0
2672,Kuwait Catches Trump In MAJOR Lie But He Refu...,kuwait catches trump in major lie but he refu...,News,"February 5, 2017",2051,0
8334,Obama Teasing Stephen Curry For ‘Clowning’ Ju...,obama teasing stephen curry for clowning just...,News,"February 4, 2016",2035,0
700,Secret Memo Shows That Every Single Congressm...,secret memo shows that every single congressm...,News,"July 31, 2017",2064,0


In [ ]:
real1=real.iloc[:int(len(real)/2),:]
real2=real.iloc[int(len(real)/2):,:]

In [ ]:
real1=clean_csv(real1,1)

Cleaning CSV.
Cleaning completed.


In [ ]:
real2=clean_csv(real2,1)

Cleaning CSV.
Cleaning completed.


In [ ]:
real=pd.concat([real1,real2])

In [ ]:
real.sample(4)

,title,text,subject,date,length,identity
6932,AmCham in China says new U.S. administration n...,amcham in china says new us administration nee...,politicsNews,"December 3, 2016",906,1
1448,U.S. Interior Dept. watchdog launches probe in...,us interior dept watchdog launches probe into ...,politicsNews,"October 2, 2017",2763,1
11924,German ban on Islamist group True Religion con...,german ban on islamist group true religion con...,worldnews,"December 19, 2017",851,1
8719,U.S. police deaths build momentum for law to t...,us police deaths build momentum for law to tre...,politicsNews,"July 18, 2016",2980,1


Dropping all unnecessary columns

In [ ]:
fake=fake[['text','identity']]
real=real[['text','identity']]

In [ ]:
fake.sample(2)

,text,identity
13083,92 yr old conservative icon dies only 4 months...,0
20660,unreal elderly hispanic trump supporter pepper...,0


In [ ]:
data1=pd.concat([real,fake],ignore_index=True)

In [ ]:
data1.tail(4)

,text,identity
43748,justice yahoo settles email privacy classactio...,0
43749,sunnistan us and allied safe zone plan to take...,0
43750,how to blow 700 million al jazeera america fin...,0
43751,10 us navy sailors held by iranian military s...,0


In [ ]:
data=pd.concat([data1,data2],ignore_index=True)

In [ ]:
data.tail(4)

,text,identity,Unnamed: 0
61810,nfl playoffs schedule matchups and odds the n...,0,18058.0
61811,macys is said to receive takeover approach by ...,0,18059.0
61812,nato russia to hold parallel exercises in balk...,1,18060.0
61813,what keeps the f35 alive david swanson is an...,1,18061.0


In [ ]:
data=data[['text','identity']]
data.reset_index()
data.tail()

,text,identity
61809,rapper ti trump a poster child for white supre...,0
61810,nfl playoffs schedule matchups and odds the n...,0
61811,macys is said to receive takeover approach by ...,0
61812,nato russia to hold parallel exercises in balk...,1
61813,what keeps the f35 alive david swanson is an...,1


Importing dependencies for ANN and Representation

In [ ]:
import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Conv1D,MaxPool1D,Dropout,GRU
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score

Tokenizing the dataset for use in the ANN

In [ ]:
def tokenize_df(df,max_features,max_len): #tokenize the df using keras tokenizer
        print('Tokenizing DF.')
        tokenizer=Tokenizer(num_words=max_features,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',split =' ') #filters is redundant, but still keeping if I missed any symbols previously
        tokenizer.fit_on_texts(texts=df['text']) #fit
        X=tokenizer.texts_to_sequences(texts=df['text'])
        X=pad_sequences(sequences=X,maxlen=max_len,padding='pre') #padding the set of words
        Y=df['identity'].values #identities (0/1)
        V=tokenizer.word_index #vocabulary
        print('Tokenized DF.')
        return X,Y,V

In [ ]:
X,Y,V=tokenize_df(data,max_features,max_len)

Tokenizing DF.
Tokenized DF.


Creating the model and compiling it to be used.

In [ ]:
def create_lstm_model(max_features): #creating the internal structure of the simple LSTM model
        print('Creating LSTM model.')
        lstm_model=Sequential(name='Sierra')
        lstm_model.add(layer=Embedding(input_dim=max_features,output_dim=128,name='Echo'))
        lstm_model.add(layer=LSTM(units=128,name='Lima'))
        lstm_model.add(layer=Dense(1,activation='sigmoid',name ='Omega'))
        print('Model creation completed.')
        lstm_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
        print('Model compiled.')
        return lstm_model

In [ ]:
lstm_model=create_lstm_model(max_features)

Creating LSTM model.
Model creation completed.
Model compiled.


In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y)

In [ ]:
lstm_model.fit(X_train,Y_train,batch_size=batch_size,epochs=epoch_count,verbose=1,validation_split=test_prop,shuffle=1,use_multiprocessing=True)

Epoch 1/100
64/64 [==============================] - 35s 497ms/step - loss: 0.5000 - accuracy: 0.7618 - val_loss: 0.2934 - val_accuracy: 0.8724
Epoch 2/100
64/64 [==============================] - 29s 454ms/step - loss: 0.2231 - accuracy: 0.9183 - val_loss: 0.2396 - val_accuracy: 0.9109
Epoch 3/100
64/64 [==============================] - 29s 448ms/step - loss: 0.1557 - accuracy: 0.9432 - val_loss: 0.2051 - val_accuracy: 0.9206
Epoch 4/100
64/64 [==============================] - 29s 461ms/step - loss: 0.3223 - accuracy: 0.8482 - val_loss: 0.3711 - val_accuracy: 0.8282
Epoch 5/100
64/64 [==============================] - 30s 469ms/step - loss: 0.2008 - accuracy: 0.9304 - val_loss: 0.2360 - val_accuracy: 0.9057
Epoch 6/100
64/64 [==============================] - 28s 446ms/step - loss: 0.1278 - accuracy: 0.9566 - val_loss: 0.2349 - val_accuracy: 0.9262
Epoch 7/100
64/64 [==============================] - 29s 449ms/step - loss: 0.1152 - accuracy: 0.9644 - val_loss: 0.1919 - val_accuracy:

In [ ]:
Y_pred=(lstm_model.predict(X_test)>=0.5).astype(int)

483/483 [==============================] - 12s 25ms/step


In [ ]:
accuracy_score(Y_test,Y_pred)

0.9437038954316035

In [ ]:
print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.95      0.94      0.95      8159
           1       0.94      0.95      0.94      7295

    accuracy                           0.94     15454
   macro avg       0.94      0.94      0.94     15454
weighted avg       0.94      0.94      0.94     15454



In [ ]:
lstm_model.save('models/ds1-2_combined_epoch-100')

In [ ]:
nmod=tf.keras.saving.load_model('models/ds1-2_combined_epoch-100')

In [ ]:
y_pred = (nmod.predict(X_test) >=0.5).astype(int)

483/483 [==============================] - 12s 25ms/step


In [ ]:
accuracy_score(Y_test,y_pred)

0.9437038954316035